In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
import time
import random
import cv2
import pandas as pd
from keras import layers, models, optimizers, losses, metrics, Sequential
from keras.layers import Dense, Conv2D, Flatten , Dropout, MaxPooling2D

In [2]:
val_train = tf.keras.utils.image_dataset_from_directory("val_images", labels="inferred", label_mode="categorical", color_mode="grayscale", batch_size=32, image_size=(100, 100), shuffle=True, seed=123, interpolation="bilinear", follow_links=False, validation_split=0.2, subset="training")
val_validate = tf.keras.utils.image_dataset_from_directory("val_images", labels="inferred", label_mode="categorical", color_mode="grayscale", batch_size=32, image_size=(100, 100), shuffle=True, seed=123, validation_split=0.2, subset="validation", interpolation="bilinear", follow_links=False)

print(val_train.class_names)
print("Sizes: ", val_train.cardinality(), val_validate.cardinality())

print("Suit Train Shape: ", val_train.element_spec)

Found 329 files belonging to 13 classes.
Using 264 files for training.
Found 329 files belonging to 13 classes.
Using 65 files for validation.


2023-04-09 16:15:30.391990: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


['10', '2', '3', '4', '5', '6', '7', '8', '9', 'ace', 'jack', 'king', 'queen']
Sizes:  tf.Tensor(9, shape=(), dtype=int64) tf.Tensor(3, shape=(), dtype=int64)
Suit Train Shape:  (TensorSpec(shape=(None, 100, 100, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 13), dtype=tf.float32, name=None))


In [3]:
# Lets make a network
# We will use a convolutional neural network

# First we will define our model
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(100,100,1)))
model.add(MaxPooling2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPooling2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPooling2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(13, activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 50, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 25, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 25, 25, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 64)       0

In [5]:
# Train the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(val_train, epochs=10, validation_data=val_validate)

Epoch 1/10
9/9 [==============================] - 2s 129ms/step - loss: 0.0964 - accuracy: 0.9811 - val_loss: 0.2618 - val_accuracy: 0.9231
Epoch 2/10
9/9 [==============================] - 1s 114ms/step - loss: 0.0785 - accuracy: 0.9811 - val_loss: 0.1076 - val_accuracy: 0.9538
Epoch 3/10
9/9 [==============================] - 1s 118ms/step - loss: 0.0202 - accuracy: 0.9962 - val_loss: 0.1000 - val_accuracy: 0.9692
Epoch 4/10
9/9 [==============================] - 1s 113ms/step - loss: 0.0202 - accuracy: 0.9886 - val_loss: 0.0667 - val_accuracy: 0.9846
Epoch 5/10
9/9 [==============================] - 1s 105ms/step - loss: 0.0148 - accuracy: 0.9924 - val_loss: 0.2209 - val_accuracy: 0.9385
Epoch 6/10
9/9 [==============================] - 1s 103ms/step - loss: 0.0355 - accuracy: 0.9848 - val_loss: 0.3409 - val_accuracy: 0.8769
Epoch 7/10
9/9 [==============================] - 1s 105ms/step - loss: 0.0417 - accuracy: 0.9924 - val_loss: 0.3990 - val_accuracy: 0.8923
Epoch 8/10
9/9 [====

In [6]:
model.save("val_model.h5")